In [3]:
import jax
import jax.numpy as jnp
from jax import random
from jax import jit, vmap, grad
from jax import scipy as jsc
import matplotlib.pylab as plt
import seaborn as sns
from scipy.stats import chi2
import numpy as np

sns.set(style="whitegrid")

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
import pandas as pd
import pymc as pm

import numpy as np
from tqdm import tqdm
import seaborn as sns

from jax.scipy.special import expit

ModuleNotFoundError: No module named 'pymc'